# Results on comparison of PBE0 runs on H2O molecules
In this notebook we will analyse the different walltimes that are performed for various calculations on Piz-Daint machine of various systems of H2O molecules.
We will only concentrate on runs which have a nspin=1 calculation and one single point of 13 iterations.
Let us first retrieve in the directory the runs that match the requirements.

First, let us uncompress the files into a separate directory:

In [1]:
import os
os.system('tar xJvf testfiles/H2O-PBE0.tar.xz')

0

In [2]:
from BigDFT import Logfiles as L
allogs=!find . -name log\*.yaml
logs=[]
for f in allogs:
    #try:
    log=L.Logfile(f)
    #except:
    #    log=None
    #    print 'Log:',f,'notcorrectly parsed'
    #logs.append(log)

Document 31 of stream NOT loaded, error: while parsing a flow mapping at line 456, column 11
did not find expected ',' or '}' at line 459, column 1
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'different runs')
('Found', 4, 'd

ValueError: No log information provided.

For any of the parsed logs let us identify which are the logs which answer to the provided criteria

In [3]:
import numpy as np
from os import path
from futile import Time as T
validlogs=[]
ns=set()
ncpus=set()
for f,l in zip(allogs,logs):
    #conditions for selection
    #print f
    if l is None or len(l) > 0 or not hasattr(l,'energy'): 
        continue
    #verify if the run if with nspin=1
    norbu,norbd=l.evals[0].info
    if norbd != 0: continue
    #get the number of WFN_OPT iterations
    nit=len(L.find_iterations(l.log)[0])-1
    if nit != 12: continue
    #get the number of molecules
    nmol=l.nat/3
    #verify if the density is constant
    cell=np.array(l.astruct['Cell'])
    vol=np.product(cell)    
    #print 'density of the system',1.0*nmol/vol
    #print 'file chosen',f
    #print 'number of iterations',nit
    radical=l.log['radical']
    dr=path.dirname(f)
    log=path.basename(f)
    timeyaml=path.join(dr,'time-'+radical+'.yaml')
    istime=path.isfile(timeyaml)
    #print 'dr',dr,'radical',radical,'time',istime
    nproc=l.log['Number of MPI tasks']
    omp=l.log['Maximal OpenMP threads per MPI task']
    nnodes=l.log['MPI tasks of root process node']
    if nproc-(nproc/nnodes)*nnodes != 0: continue
    #print l.log['dft']['ixc']
    data={'Run':l, 'label':radical,'directory':dr,
          'nproc':nproc,'omp':omp,'nnodes':nproc/nnodes,'xc':l.log['dft']['ixc'],
         'n':nmol}
    if istime: data['Time']=timeyaml
    validlogs.append(data)
    ns.add(nmol)
    ncpus.add(nproc/nnodes)
ns=list(ns)
ncpus=list(ncpus)
ns.sort()
ncpus.sort()
print 'number of molecules:',ns,'number of nodes:',ncpus

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


number of molecules: [] number of nodes: []


And then let us sort them in increasing order:

In [11]:
orderedlogs=[]
for xc in [-101130,-406]:
    for n in ns:
        for nc in ncpus:
            for l in validlogs:
                if l['nnodes']==nc and l['n']==n and l['xc']==xc:
                    orderedlogs.append(l)

In [12]:
def select(dict_selection,validlogs):
    selection=[]
    for l in validlogs:
        found=True
        for k in dict_selection:
            if k not in l or l[k] != dict_selection[k]: found=False
        if not found: continue
        selection.append(l)
        #print l['n'],l['xc'],l['nnodes']
    return selection

Let us now consider, for each of the system's sizes, all the different comparisons we have.
We group together the calculations with the same number of nodes:

In [9]:
allcomp=[]
for n in ns:
    for nn in ncpus:
        runs=select({'nnodes':nn,'n':n},orderedlogs)
        if len(runs)>0:
            perf=[r['Time'] for r in runs]
            times=T.TimeData(*perf,fontsize=8)
            data=[(r['n'],r['xc'],r['nnodes'],tots) for r,tots in zip(runs,times.totals)]
            print 'new comparison',len(runs),n,nn
            #print data
            allcomp.append([data,perf])

With commands like the following we obtain the plot to compare the different runs with a given number of molecules with each other:

In [10]:
#%matplotlib auto
T.TimeData(*allcomp[0][1],title='64 Molecules').show()

IndexError: list index out of range

We might further select for each of the category the minimum value for all the different runs:

In [9]:
mincomp=[]
for runn in allcomp:
    if len(runn[0])==2: 
        mincomp.append(runn)
        continue
    tminPBE=1.0e9
    tminPBE0=1.0e9
    for i,l in enumerate(runn[0]):
        t=l[-1]
        xc=l[1]
        #print 'test',t,xc
        if t<tminPBE and xc==-101130:
            tminPBE=t
            PBE=l
            PBEfile=runn[1][i]
        elif t<tminPBE0 and xc==-406:
            tminPBE0=t
            PBE0=l
            PBE0file=runn[1][i]
    mincomp.append([[PBE,PBE0],[PBEfile,PBE0file]])


In [10]:
%matplotlib auto
T.TimeData(*mincomp[0][1],title='64 Molecules').show()

Using matplotlib backend: Qt4Agg
Good bye!


Now we can obtain a table with the values of the best results, including the gamma, defined as the ratio bw a PBE0 run  walltime and a PBE one:

In [13]:
print '2x (nmols, xc, nnodes, walltime), gamma'
for manip in mincomp:
    val=manip[0]
    #for n,xc,nn,t in manip[0]:
    #    print n,t
    print val,val[1][-1]/val[0][-1]

2x (nmols, xc, nnodes, walltime), gamma
[(64, -101130, 32, 11.0), (64, -406, 32, 24.0)] 2.18181818182
[(128, -101130, 64, 23.0), (128, -406, 64, 74.0)] 3.21739130435
[(256, -101130, 128, 49.0), (256, -406, 128, 244.0)] 4.97959183673
[(256, -101130, 256, 36.0), (256, -406, 256, 200.0)] 5.55555555556
[(512, -101130, 256, 115.0), (512, -406, 256, 1030.0)] 8.95652173913
[(512, -101130, 512, 79.0), (512, -406, 512, 798.0)] 10.1012658228


Where we see that for 512 molecules gamma is better with 256 nodes, although walltime is higher of course.

We may then also remove the directory that the tarfile created:

In [15]:
import shutil
shutil.rmtree('degomme')